I want this to be the single best and most complete document that exists on how to make money on cryptocurrency exchanges using triangular arbitrage. This is a labour of love—lots of trial and error and experience went into getting this knowledge.

# Roadmap

1. Absolute Basics - What is Triangular Arbitrage?
2. Algorithmic Solution
3. Implementation Details
4. Speed tricks

# Basics

Let's break down the term triangular arbitrage. First, arbitrage is a fancy word for buying low and selling high.

Pretend you are walking down the street and you see an open farmer's market where people are buying and selling bread for \\$1 per loaf. You keep walking and 5 minutes later you see another market this time selling bread for \\$2 per loaf. Displaying unrivalled business acumen, you decide to head back and buy as many \\$1 loaves as your greedy little hands can carry. You bring them to the new market, sell them for \\$2 and make a clean buck for each loaf.

You just completed bread arbitrage.

And made every finance researcher angry. Because many financial theories assume that this situation won't happen. They assume that if there is a price difference in equivalent assets (same bread), then smart individuals like yourselves will buy cheap and sell at the expensive place until the prices equal out. Imagine if whole hordes of people went to the \\$1 place, trying to buy as many loaves as possible. The sellers would quickly raise their prices.

And equilibrium is reached when both markets have the same price, \\$1.50 per loaf.

Let's make this example a little more relevant to our future topic.

Pretend that you own 1 USD. And you can buy 1.5 CAD with 1 USD on Bob's exchange. Jim's exchange will let you sell 1.5 CAD for 1.1 USD. Now you buy 1.5 CAD on Bob's exchange and immediately sell it on Jim's exchange. You end up with 1.1 USD.

You made 10 cents for free! This is currency arbitrage.

### Triangular?

Now we get arbitrage. Where does the triangular part come in?

Well, it means doing our currency arbitrage like above but using 3 different currencies instead of just USD and CAD. Here's an example.

Exchange Name | Trade Available
--- | ---
Bob's | 1 USD for 1.5 CAD
Jim's | 1.5 CAD for 2 Euros
Ted's | 2 Euros for 1.1 USD

Now we buy CAD at Bob's exchange, trade that for Euro's at Jim's and go back to USD at Ted's, making a sweet 10 cents.

This seems crazy simple. And it is... when I've laid it out so nicely for you. Things get more complicated when looking at real exchange data because you have to keep in mind exchange ratios.

As a reminder, if you have 1 USD to 1.5 CAD, you can find it in terms of 1 CAD by dividing by 1.5. so you get 0.67 USD for 1 CAD. It's helpful to think of it as a ratio—1.5 CAD to 1 USD. Then divide both sides by 1.5 to get 1 CAD to 0.67 USD.

Now let's work through the full example.

_        | USD  | CAD  | Euro
---      | ---  | ---  | ---
**USD**  | 1    | 1.5  | 1.9
**CAD**  | 0.67 | 1    | 1.33
**Euro** | 0.55 | 0.75 | 1

This table means we can trade 1 USD for 1.5 CAD or 1.9 Euro. And we can trade 1 Euro for 0.55 USD and 0.75 CAD.

In this example, starting with USD we go 1 USD for 1.5 CAD. That 1.5 CAD is traded for 1.5 $\times$ 1.33 = 1.995 Euros. The 1.995 Euros are traded for 1.995 $\times$ 0.55 = 1.10 USD.

But how did I know that USD to CAD to Euros to USD was going to make money? Why not USD to Euro to CAD to USD? Does the order matter? Well the simplified math of USD to Euro to CAD to USD is 1 $\times$ 1.9 $\times$ 0.75 $\times$ 0.67 = 0.95 USD. You lost 5 cents on the round trip.

Now, typical exchanges have tons of currencies. Instead of a 3x3 table, you have a 100x100 table. How do you know which series of trades is going to work, if any? What if there is some crazy series of 10 trades in a cycle?

# Algorithmic Analysis

This is an excellent problem for a computer.

Now, if you have any programming experience you probably have an idea of how to solve this. Most people's first thought is that you just find all possible 3 currency combinations and then calculate if the exchange rates multiply to greater than one. And that was my first thought too. My very first bot did 3 loops checking for each 3 currency combo. I used Quadriga, a Canadian cryptocurrency exchange that has since gone out of business. And this worked fine because Quadriga only had a few currencies. They only had CAD, USD, bitcoin, litecoin, and ethereum. But then I wanted to collect data and see which exchange would be most profitable. I remember looking at Binance, the largest exchange at the time, which had hundreds of different altcoins. At some point I remembered being disappointed by the results. Something spurred me on to try looking at longer trade sequences where you would buy and sell 4 different currencies. This consisted of me just adding another loop to my code. And I got better results! It was finding more profitable trades. This surprised me. I thought a 4 currency combo was a total shot in the dart because exchanges charge fees on each trade. So trading 4 currencies you spend more fees than trading 3 currencies. I thought that the extra fees would eat any arbitrage sequences. I was wrong. Big time. I then started looking at longer paths. I kept finding profitable trades of more currencies, with some using 7 or 8! Now almost all profitable trades I found were between more than 3 coins. So we need to find all 7 currency combinations and then check each. But what if there is an 8 sequence trade? How do we know we have found all of them? If you have 100 currencies and want to make groups of 7, there are 16007560800 total combinations. I also wanted to look at the exchange Hitbtc and it had something like 400 cryptocurrencies. The amount of combinations there is massive. What is the most efficient way to do all these calculations?

My program was getting unwieldy with 10 loops and was getting super complicated. And then when I took an algorithms class I realized this problem can be formulated as a graph search problem. 

We can visualize the problem as a weighted graph. Each currency corresponds to a node, while the exchange rates correspond to edges. The weight of an edge from node $u$ to $v$ is the amount of $v$ we can buy with 1 $u$. Then an arbitrage opportunity exists if we find a cycle in the graph where the edges multiply to more than 1.

Now we can use graph shortest path algorithms to find this cycle. But there is a problem. The shortest path algorithms assume the edges are summed together. Triangular arbitrage is multiplicative. We calculate the total amount of USD we have at the end by multiplying all the exchange rates together.

So to turn multiplication into addition, we set the edges to be the log of the exchange rate.

The other problem is that we are trying to find the **longest** path. We assume that most series of trades will not be profitable and will result in a final trip value of less than 1. We want the trades that result in the largest value. Thus we set edges to be the negative log of exchange rates. Now shortest path algorithms will find the smallest value, which will be our most profitable trade.

Here's that logic in formulas:

When $R_{ij}$ is the exchange rate between currency $i$ and $j$ then we have arbitrage if

$$R_{1,2} \times R_{2,3} \times R_{3,4} \times \ldots \times R_{k-1,k} \times R_{k,1} > 1$$

$$\log R_{1,2} + \log R_{2,3} + \log R_{3,4} + \ldots + \log R_{k-1,k} + \log R_{k,1} > 0$$

$$(-\log R_{1,2}) + (-\log R_{2,3}) + (-\log R_{3,4}) + \ldots + (-\log R_{k-1,k}) + (-\log R_{k,1}) < 0$$

As we are finding the shortest path in a graph with negative edge weights, the bellman-ford algorithm is the best choice. It finds the shortest path to all nodes from a starting node and can detect cycles where the sum of the edges is a negative value. The running time of bellman-ford is O(|V||E|). Here |E| is O($n^2$) where $n$ is the number of nodes so the total time complexity is O($n^3$). So we can find profitable cycles of **any** length in the equivalent of three loops. 

## Bellman-Ford

Bellman-Ford sets the distance to the starting node to 0. Then all other nodes get a distance of infinity. Then it looks at all edges and checks if the distance to the node connected to the edge is shorter if it takes the edge. Then it updates the distance. Now it has all the shortest paths of length 1. And at each iteration i, it scans all edges and finds all the new shortest paths of at most length i. It does this for |V| - 1 times as that is the longest possible path with no cycle. Then it checks one more time. If any distances are updated then it knows that the path is length |V|, which could only occur if there is a negative cycle.

We use a list of nodes and a list of edges. The edge ($u$, $v$, $w$) is the weight from node $u$ to node $v$. We also add a dummy node with a link to all nodes and use it as the start node to make sure we can detect negative cycles not reachable if we were to pick an existing node as the start. This probably isn't necessary as the graphs you use in currency exchange should be connected—it should be possible to get to any currency from any other currency. But just in case I added this step.

Then we modify the algorithm slightly to keep track of the paths. We don't just want to know that a negative cycle exists, we want to know what it is. To do this we keep track of the predecessor to a node in the shortest path, then when we find a negative cycle we loop back through the predecessors until we get to a repeated node.

This will output the first negative cycle found, you can change it to print out all negative cycles. You just have to filter out the same cycle in different orders. For example, \['EUR', 'USD', 'CAD'\] and \['USD', 'CAD', 'EUR'\] are the same series of trades.

In [45]:
import math
nodes = ['CAD', 'USD', 'EUR']
edges = [['USD', 'CAD', 1.5],
         ['USD', 'EUR', 1.9],
         ['CAD', 'USD', 0.67],
         ['CAD', 'EUR', 1.33],
         ['EUR', 'USD', 0.55],
         ['EUR', 'CAD', 0.75]]

def bellman_ford(nodes, edges):

    for edge in edges:
        edge[2] = -math.log(edge[2])
    
    for node in nodes:
        edges.append(('dummy', node, 0))
    
    nodes.append('dummy')
    
    distance = dict.fromkeys(nodes, float('inf'))
    distance['dummy'] = 0
    
    predecessor = dict.fromkeys(nodes)

    for _ in range(len(nodes) - 1):
        for (u, v, w) in edges:
            if distance[u] + w < distance[v]:
                distance[v] = distance[u] + w
                predecessor[v] = u

    for (u, v, w) in edges:
        if distance[u] + w < distance[v]:
            cycle = [v]
            last = predecessor[v]
            while last not in cycle:
                cycle.append(last)
                last = predecessor[last]
            cycle.reverse()
            return cycle

    return 'No arbitrage possible'

bellman_ford(nodes, edges)

['EUR', 'USD', 'CAD']

# Implementation Details

### Fees and Spread

Okay, so we've got a running algorithm. But we are not quite ready to head out into the world and use it. There's a lot of specifics that make things tricky.

You need to worry about the bid-ask spread. In the example above we assumed there was just a price listed. It doesn't work like that. Exchanges have an order book. It is a sorted list of orders people have previously placed. So an order book for bitcoin might look like:

Bids | Asks
--- | --- 
Price, Amount | Price, Amount
\\$7500, 0.1 BTC | \\$7600, 0.3 BTC
\\$7450, 10 BTC | \\$7620, 5 BTC
\\$7420, 2 BTC | \\$7700, 10 BTC

This means that 3 people want to buy bitcoin. The highest offer (bid) is for an exchange rate of \\$7500 per bitcoin, and that person wants to buy 0.1 BTC. The next highest offer is for \\$7450 per bitcoin and they want to buy 10 BTC.

Similarly, 3 people want to sell bitcoin. The lowest selling offer (ask) is \\$7600 per bitcoin, and they want to sell 0.3 BTC.

Let's work through an example. Imagine that Jim wants to buy 1 bitcoin and is willing to pay up to \\$7600 for it. Then when he makes his \\$7600 buy order, he is matched with the \\$7600 sell offer and the deal is done. But that person only wanted to sell 0.3 bitcoin, leaving part of Jim's order unfilled.

Then the order book will look like this:

Bids | Asks
--- | --- 
Price, Amount | Price, Amount
\\$7600, 0.7 BTC | \\$7620, 5 BTC 
\\$7500, 0.1 BTC | \\$7700, 10 BTC
\\$7450, 10 BTC | 
\\$7420, 2 BTC | 

Thus the spread refers to the difference between the bid and ask prices. If you want to buy or sell bitcoin immediately, you have to buy at a slightly higher price and sell at a slightly lower price. This means that there is not just one price of bitcoin. Usually when you see people talk about **the** price of bitcoin, they take the average of the best bid and best ask. This is called the mid-price. But it's not a price that you can trade at. Unless you make an offer, let it sit there, and hope someone else fills it.

When you put an offer in the order book at a specific price and let it sit there, that is called a limit order and you are called the market maker. When you match with a previously posted offer you are called a market taker.

The next problem is the fees. Every exchange charges fees on each trade. So now you need to make the edges of the graph the exchange rate, after fees. This complicates the math. Especially since most exchanges have complicated fee structures. They often charge different levels of fees depending on how much you trade and charge different fees depending on if you are the market maker vs market taker. 

### How much to Trade

If you just run the above algorithm on some cryptocurrency exchange you might get some hits. But you have to be careful because you have to know how much to make the trade for. The orders you plan to fill won't all be the same size. You have to find the highest common liquidity—the order with the lowest real dollar value. The math gets messy as you have to convert each order into a common currency and compare them. 

### Minimum Trade Amounts

I remember looking at HitBTC and the minimum order sizes of each currency varied drastically. One might have a min of \\$100 USD equivalent and the other a min of \\$1 USD. This means that even when you find the lowest common liquidity, sometimes you can't execute it because you can't trade that small in one currency. So then you have to store all the mins and check for those.

### Order precisions

Some exchanges have extremely coarse-grained order precision. For example, I think on HitBTC you were limited to trades of increments in 0.01 BTC. So if your bot wants to buy 0.055 BTC you are forced to buy either 0.06 or 0.05 BTC. Now you can't buy the same amount of each currency.

You can still make arbitrage money even if the exact amounts don't line up. Look it like making the arb trade, and then just randomly also trading a small amount of a different coin. On average you shouldn't be worse off by trading assets at market price. But this is something you have to consider when taking the tri arb sequence and sending in the orders. 

### How much profit do you pull the trigger on?

I set a parameter in my bot that decided how profitable the trade needed to be to execute it. The more profit you demand, the fewer trades you make. But if you don't have a profit buffer than you can end up making trades that lose a slight amount of money just because of rounding issues. This parameter will depend on the exchange.

# Need for Speed

Up until now this has been a bit theoretical. But if you do this you must realize that you are in a race. You vs the world. When I first started doing this I would first just log if there were any profit opportunities. Then I started logging how long they lasted. It was never longer than a second, usually closer to half a second. Just a blink of an eye. I realized there was money sitting out there waiting for me. I just had to take it. I had to be fast enough to take it.

Because obviously someone else had gotten there first. Someone else was doing the same thing I was and eating up the profits. I decided to try and race them. I was always reminded of playing the pokemon games on my gameboy as a kid. No matter how fast I went, my rival was always one step ahead.

### Going Faster

An exchange receives an order and updates its order book. They send me this information. About 100ms later we get it through the internet. Our program does its calculations in 2ms and sends orders to the exchange. 100ms later the exchange gets the orders. So the big time sink here is network latency.

The first thing I did was to run my bot on a server instead of my home computer. That got my ping down quite a bit. I tried a few different hosts to see which one would have the lowest ping. The problem is that exchanges use Cloudflare, a CDN. You might get sent to one of many different servers as your initial contact point—it's trying to hit a moving target. I just used trial and error to find a server with low latency.

But then I realized that it takes a long time for the exchange to process an order. Sometimes up to a second. So if you trade USD for BTC, wait for the BTC, then trade that, it's already been 2 seconds and your profits are gone. You are left with the bitter taste of your rival's dust. 

The only way to go fast enough is to hold **all** currencies, waiting in reserve, and then make all the trades simultaneously. This makes a huge difference in speed and is a necessity unless the arb opportunities are sitting there for seconds. Doing 7 trades while waiting for a confirmation on each one would be like molasses compared to my rival.

### Nonce errors

You run into a problem when you try to do so many simultaneous trades. Each exchange API usually requires a nonce—an integer number that goes up by one each API call you make. So when you send a bunch of trades all in a row without waiting for your response, you don't know in what order they will arrive. This is because they can get routed through the internet in different ways. Even if you send them order 1, 2, 3, they can show up 3, 1, 2. So your nonces are going to be wrong from the exchange's point of view. So you have to make a bunch of API keys, and use a different one for each trade, with a different nonce for each key. 

## Risk-Free?

The whole point of arbitrage is that it is supposed to be risk-free, easy money. But now you have to hold as many currencies as you want to trade between. Holding a portfolio of 100 cryptocurrencies is ultra risky. And small altcoin prices can shoot up and down a lot, leaving the value of your portfolio unclear on a day to day basis. Figuring out the dollar value of your portfolio is surprisingly complicated. It is difficult to tell if you are even making money because your bot will just be swapping between 100 currencies. So you have to take snapshots of portfolio amounts, and convert that all into a cash value periodically to track your earnings. I recommend storing your portfolio values at specific periods so you can do a comparison between what the value of your portfolio would have been if you did no arbitrage. This will tell you the true profits of your bot.

### Reduce your Crypto Exposure

If you want to limit your exposure to crypto while still making money from arbitrage you should go short on cryptocurrency. Going short means betting against it.

Almost all cryptocurrency prices are heavily correlated to bitcoin. So to remove all your risk you calculate the bitcoin value of your portfolio and short that much bitcoin. Now when crypto prices go down, your short investment should profit as much as your currencies lose. But shorting bitcoin opens up a whole other can of worms as the interest you pay is unpredictable. I remember it one time it jumped from around from 6% annually to 40%. Now you make or lose money based on the bitcoin interest rate.

For a while I tried to get super fancy. I used a topic from finance and calculated the beta of lots of different cryptocurrencies compared to bitcoin. You compare the prices and try and find how correlated they are. If a cryptocurrency has a beta of 2 compared to BTC, that means that if bitcoin goes up 10\%, this coin is likely to go up 20\% at the same time. I tried to calculate the overall beta of my portfolio. That would tell me how correlated it was with the bitcoin price and short precisely that much bitcoin so that my exposure evened out. So if my portfolio beta was 0.9, and worth \\$1000, then I should short \\$900 BTC to cancel out currency fluctuations. But this proved ineffective in practice. My portfolio was changing so often that the extra work wasn't worth it. The beta of my portfolio wasn't stable over time and it was a hassle trying to get the right hedge.

Because of these problems I found that my portfolio was still extremely volatile. I thought about trying to short different currencies in proportion to my holdings, but that gets even more complicated.

### Limit vs Market Orders?

One big decision you have to make is whether to do limit or market orders.

A limit order posts the order for a specific amount, while a market order just matches whatever the best offer is. Market orders are simpler to make, you don't specify a price.

But if you are late—your rival beats you to the punch and takes that profitable order off the book—a market order will go through and make the deal with the next best offer. And that results in a loss.

And you can't beat your rival every time. Even if you are the fastest arbitrage bot, you will still lose sometimes because of the inherent delay in the system. When you get a price update, that is a snapshot in time from 100ms ago. But more orders have gone in during the last 100ms and maybe your arb order is off the books already. Then you do your calculations and send in your orders. Your orders get there in 100ms and take a few hundred ms before they are processed. In all that time someone else might have taken away your arb trades by accident. 

I found that a much better strategy was to just do limit orders at the same price to match the other limit order. Then if someone stole the order from me, my limit order would just sit open. And you have a choice. (1) You can cancel it and just accept that you lost money because of trading fees. Or (2) you let it sit there and hope it closes eventually. I started just letting them sit there and found that most of the time the order would close within a few days. Because cryptocurrencies were so volatile, the price would usually bounce to where I needed it.

One risk I thought I would face is that if you are leaving all these open orders then your cryptocurrencies will be out of balance. But it never became a big deal. The orders that didn't fill immediately seemed to be randomly distributed and thus my coins stayed fairly balanced.

### Multiple IPs

I never got around to implementing this, but it is something that I considered doing.

Some exchanges only offered an HTTP API, but most offer WebSockets now. Exchanges will rate limit HTTP APIs. This prevents you from getting the most accurate data. You can use multiple IP's to get around that. 

Websockets give real-time data updates, but connection speeds still vary. Exchanges operate a list of WebSocket connections so the data gets sent out to each user one by one. If you are up further in the list then you will get the data sooner then your rival. This is random and hard to control. But you can make a bunch of WebSocket connections and test the speed of each by recording the time you get the same data. Then you drop the slow ones and just use the fast ones. But most exchanges won't like it if you open a ton of WebSocket connections, so they either shut them down or just don't respond to some.

But if you create these connections from separate IP addresses you will be in the clear. 

# Why did I stop?

This is all past tense. I don't have any tri-arb bots running at the moment. They stopped working. Triangular arbitrage profits from market inefficiencies. Over time it became harder and harder to get the trade first. My percentage of immediately filled orders went down. And if the prices of currencies stay steady then the profits are quickly picked up and nothing else happens. It is only if prices wildly swing then it creates slight delays for all the prices to get on board with the changes. Arbitrage strategies are best when prices are very volatile. So my bot would sit there silently for a few days, waiting for arb opportunities, and only making money during big price swings.

Holding crypto is risky. It would only be worth it for very large returns.